In [1]:
!pip install lark-parser > /dev/null

import re
from lark import Lark, Transformer


 LEXER

In [2]:
KEYWORDS = {"int", "print", "if", "else", "while", "main"}
OPERATORS = {"==","!=",">=","<=","+","-","*","/","=","<",">"}
PUNCTS = {";", ",", "(", ")", "{", "}"}

_token_spec = [
    ("COMMENT",  r"//[^\n]*"),
    ("NUMBER",   r"\d+"),
    ("ID",       r"[A-Za-z_][A-Za-z0-9_]*"),
    ("OP",       r"==|!=|>=|<=|[+\-*/=<>]"),
    ("PUNCT",    r"[;,(){}]"),
    ("SKIP",     r"[ \t\r\n]+"),
    ("MISMATCH", r"."),
]
_tok_re = re.compile("|".join(f"(?P<{n}>{p})" for n, p in _token_spec))

def tokenize_summary(code: str):
    cats = {"Keyword":[], "Identifier":[], "Constant":[], "Operator":[], "Punctuation":[], "Comment":[]}
    for m in _tok_re.finditer(code):
        k = m.lastgroup; v = m.group()
        if k == "COMMENT": cats["Comment"].append(v); continue
        if k == "SKIP":    continue
        if k == "NUMBER":  cats["Constant"].append(v); continue
        if k == "ID":
            (cats["Keyword"] if v in KEYWORDS else cats["Identifier"]).append(v); continue
        if k == "OP":      cats["Operator"].append(v); continue
        if k == "PUNCT":   cats["Punctuation"].append(v); continue
        if k == "MISMATCH": raise ValueError(f"Unexpected char: {v}")
    return cats


SYNTAX ANALYZER (Parser + AST)

In [3]:
GRAMMAR = r"""
    start: "int" "main" "(" ")" "{" stmt* "}"

    ?stmt: decl ";"
         | assign ";"
         | print_stmt ";"
         | if_stmt
         | while_stmt
         | "{" stmt* "}"        -> block

    decl: "int" CNAME ("=" expr)?
    assign: CNAME "=" expr
    print_stmt: "print" "(" expr ")"
    if_stmt: "if" "(" expr ")" stmt ("else" stmt)?
    while_stmt: "while" "(" expr ")" stmt

    ?expr: logic
    ?logic: rel
          | logic "==" rel   -> eq
          | logic "!=" rel   -> ne
    ?rel: add
        | rel ">"  add       -> gt
        | rel "<"  add       -> lt
        | rel ">=" add       -> ge
        | rel "<=" add       -> le
    ?add: mul
        | add "+" mul        -> add
        | add "-" mul        -> sub
    ?mul: atom
        | mul "*" atom       -> mul
        | mul "/" atom       -> div
    ?atom: NUMBER            -> number
         | CNAME             -> var
         | "(" expr ")"

    %import common.CNAME
    %import common.NUMBER
    %import common.WS
    %ignore WS
"""

class ASTBuilder(Transformer):
    def number(self, n): return ("num", int(n[0]))
    def var(self, n):    return ("var", str(n[0]))
    def add(self, i):    return ("add", i[0], i[1])
    def sub(self, i):    return ("sub", i[0], i[1])
    def mul(self, i):    return ("mul", i[0], i[1])
    def div(self, i):    return ("div", i[0], i[1])
    def eq(self, i):     return ("eq",  i[0], i[1])
    def ne(self, i):     return ("ne",  i[0], i[1])
    def gt(self, i):     return ("gt",  i[0], i[1])
    def lt(self, i):     return ("lt",  i[0], i[1])
    def ge(self, i):     return ("ge",  i[0], i[1])
    def le(self, i):     return ("le",  i[0], i[1])
    def decl(self, i):   return ("decl", str(i[0]), i[1] if len(i)==2 else None)
    def assign(self, i): return ("assign", str(i[0]), i[1])
    def print_stmt(self, i): return ("print", i[0])
    def block(self, items):  return ("block", list(items))
    def if_stmt(self, items): return ("if", items[0], items[1], items[2] if len(items)==3 else None)
    def while_stmt(self, items): return ("while", items[0], items[1])

_parser = Lark(GRAMMAR, parser="lalr")

def parse_to_ast(code: str):
    tree = _parser.parse(code)
    ast = ASTBuilder().transform(tree)
    return list(ast.children)




SEMANTIC ANALYZER

In [4]:

class Symbol:
    def __init__(self, name, typ="int"):
        self.name = name
        self.typ = typ
        self.size = {"int":4,"float":4,"double":8,"char":1}.get(typ,0)
        self.lines_decl = []
        self.lines_use = []

class SymbolTable:
    def __init__(self): self.table = {}
    def insert(self, name, typ="int", line=0):
        if name in self.table: raise RuntimeError(f"Duplicate declaration: {name}")
        s = Symbol(name, typ); s.lines_decl.append(line); self.table[name] = s
    def lookup(self, name, line=0):
        s = self.table.get(name)
        if s and line: s.lines_use.append(line)
        return s
    def dump(self):
        return [(i,n,s.typ,s.size,s.lines_decl,s.lines_use) for i,(n,s) in enumerate(self.table.items())]

def first_pass_decls(ast, st):
    for i, s in enumerate(ast, 1):
        if s[0] == "decl": st.insert(s[1], "int", i)

IR GENERATOR (TAC)

In [5]:
_tmp,_lbl = 0,0
def _new_tmp(): global _tmp; t=f"t{_tmp}";_tmp+=1;return t
def _new_lbl(): global _lbl; L=f"L{_lbl}";_lbl+=1;return L
def _reset(): global _tmp,_lbl;_tmp=_lbl=0

def gen_expr(n,st):
    k=n[0]
    if k=="num": t=_new_tmp(); return [f"{t} = {n[1]}"], t
    if k=="var": t=_new_tmp(); return [f"{t} = {n[1]}"], t
    if k in ("add","sub","mul","div","gt","lt","ge","le"):
        op={ "add":"+","sub":"-","mul":"*","div":"/",
             "gt":">","lt":"<","ge":">=","le":"<="}[k]
        c1,t1=gen_expr(n[1],st); c2,t2=gen_expr(n[2],st)
        t=_new_tmp(); return c1+c2+[f"{t} = {t1} {op} {t2}"],t

def gen_stmt(n,st):
    k=n[0]
    if k=="decl":
        name,init=n[1],n[2]
        if init is None: return [f"{name} = 0"]
        c,t=gen_expr(init,st); return c+[f"{name} = {t}"]
    if k=="assign":
        name,e=n[1],n[2]
        c,t=gen_expr(e,st); return c+[f"{name} = {t}"]
    if k=="print":
        c,t=gen_expr(n[1],st); return c+[f"print {t}"]
    if k=="if":
        cond,then,els=n[1],n[2],n[3]
        c,t=gen_expr(cond,st); L1,L2=_new_lbl(),_new_lbl()
        out=c+[f"ifFalse {t} goto {L1}"]+gen_stmt(then,st)+[f"goto {L2}",f"label {L1}"]
        if els: out+=gen_stmt(els,st)
        out+=[f"label {L2}"]; return out
    if k=="while":
        cond,body=n[1],n[2]; L1,L2=_new_lbl(),_new_lbl()
        c,t=gen_expr(cond,st)
        out=[f"label {L1}"]+c+[f"ifFalse {t} goto {L2}"]+gen_stmt(body,st)+[f"goto {L1}",f"label {L2}"]
        return out
    if k=="block":
        out=[]; [out.extend(gen_stmt(s,st)) for s in n[1]]; return out

def gen_program(ast,st):
    _reset(); code=[]
    for s in ast: code+=gen_stmt(s,st)
    return code


OPTIMIZER

In [6]:
def peephole_const_move(tac):
    out=[]; i=0
    while i<len(tac):
        if i+1<len(tac):
            p1=tac[i].split(); p2=tac[i+1].split()
            if len(p1)==3 and p1[1]=="=" and p1[2].isdigit() and len(p2)==3 and p2[1]=="=" and p2[2]==p1[0]:
                out.append(f"{p2[0]} = {p1[2]}"); i+=2; continue
        out.append(tac[i]); i+=1
    return out


CODE GENERATOR (Assembly-like)

In [7]:
def tac_to_assembly(tac):
    asm=[]
    for l in tac:
        p=l.split()
        if not p: continue
        if p[0]=="print": asm.append(f"PRINT {p[1]}")
        elif p[0]=="label": asm.append(f"LABEL {p[1]}")
        elif p[0]=="goto": asm.append(f"JMP {p[1]}")
        elif p[0]=="ifFalse": asm.append(f"JZ {p[1]} {p[3]}")
        elif "=" in p:
            dst=p[0]
            if len(p)==5: _,_,a,op,b=p; asm += [f"LOAD {a}",f"LOAD {b}",f"{op.upper()}",f"STORE {dst}"]
            elif len(p)==3: _,_,a=p; asm += [f"LOAD {a}",f"STORE {dst}"]
    return asm


DRIVER CODE

In [8]:

source_code = """int main(){
  int x = 10;
  int y = 20;
  if (x < y) { print(x); } else { print(y); }
}"""

print("=== SOURCE CODE ===\n", source_code)

# Lexical Analysis
cats = tokenize_summary(source_code)
print("\n=== TOKENS ===")
for k,v in cats.items():
    print(f"{k} ({len(v)}): {', '.join(v) if v else '-'}")

# Syntax → AST
print("\n=== AST ===")
ast = parse_to_ast(source_code)
for i,n in enumerate(ast,1):
    print(i, n)

# Semantic → Symbol Table
print("\n=== SYMBOL TABLE ===")
st = SymbolTable()
first_pass_decls(ast, st)
for row in st.dump():
    print(f"ID={row[0]}, Name={row[1]}, Type={row[2]}, Size={row[3]} bytes, Decl={row[4]}")

# IR Generation
print("\n=== IR CODE ===")
ir = gen_program(ast, st)
print("\n".join(ir))

# Optimization
print("\n=== OPTIMIZED IR ===")
opt = peephole_const_move(ir)
print("\n".join(opt))

# Assembly-like Code
print("\n=== ASSEMBLY CODE ===")
asm = tac_to_assembly(opt)
print("\n".join(asm))




=== SOURCE CODE ===
 int main(){
  int x = 10;
  int y = 20;
  if (x < y) { print(x); } else { print(y); }
}

=== TOKENS ===
Keyword (8): int, main, int, int, if, print, else, print
Identifier (6): x, y, x, y, x, y
Constant (2): 10, 20
Operator (3): =, =, <
Punctuation (18): (, ), {, ;, ;, (, ), {, (, ), ;, }, {, (, ), ;, }, }
Comment (0): -

=== AST ===
1 ('decl', 'x', ('num', 10))
2 ('decl', 'y', ('num', 20))
3 ('if', ('lt', ('var', 'x'), ('var', 'y')), ('block', [('print', ('var', 'x'))]), ('block', [('print', ('var', 'y'))]))

=== SYMBOL TABLE ===
ID=0, Name=x, Type=int, Size=4 bytes, Decl=[1]
ID=1, Name=y, Type=int, Size=4 bytes, Decl=[2]

=== IR CODE ===
t0 = 10
x = t0
t1 = 20
y = t1
t2 = x
t3 = y
t4 = t2 < t3
ifFalse t4 goto L0
t5 = x
print t5
goto L1
label L0
t6 = y
print t6
label L1

=== OPTIMIZED IR ===
x = 10
y = 20
t2 = x
t3 = y
t4 = t2 < t3
ifFalse t4 goto L0
t5 = x
print t5
goto L1
label L0
t6 = y
print t6
label L1

=== ASSEMBLY CODE ===
LOAD 10
STORE x
LOAD 20
STORE y
LO